In [8]:
from fastapi import FastAPI
import pandas as pd
from sklearn.cluster import KMeans

app = FastAPI()

#base de datos
Base_Cluster_Est = pd.read_csv("../../Datos/Raw/BaseDatos_Cluster.csv")
identificadores = pd.read_csv("../../Datos/Raw/identificadores.csv")
Base_filtrada = pd.read_csv("../../Datos/Raw/Base_filtrada.csv")

if 'Unnamed: 0' in Base_Cluster_Est.columns:
    Base_Cluster_Est.drop(columns=['Unnamed: 0'], inplace=True)

#cluster 
numero_clusters = 6
kmeans = KMeans(n_clusters=numero_clusters, random_state=42)
clusters = kmeans.fit_predict(Base_Cluster_Est)

Base_Cluster_Est['cluster'] = clusters

#medias y eso 
cluster_means = Base_Cluster_Est.groupby('cluster').mean().round(2)
centroids = pd.DataFrame(kmeans.cluster_centers_).round(2)

#merge
Base_Cluster_fusionada = Base_Cluster_Est.merge(Base_filtrada, left_index=True, right_index=True, how='right')

@app.get("/")
async def root():
    return {
        "message": "¡Bienvenido! Aquí tienes una lista de funciones disponibles y los comandos para invocarlas:",
        "functions": {
            "Obtener información de un contrato por índice": ".../indice/{index}",
            "Obtener información de un cluster por número de cluster": ".../cluster_info/{cluster}",
            "Obtener el número de fila de un contrato por id_contrato": ".../fila/{id_contrato}",
            "Comparar las características de los clusters": ".../comparar_clusters"
        }
    }

@app.get("/indice/{index}")
async def get_contract_info(index: int):
    try:
        
        contrato = Base_Cluster_fusionada.loc[index]
        
        cluster = int(contrato['cluster'])
        
        
        contrato_filtrado = contrato[Base_filtrada.columns]
        
        
        response = {
            "Índice": index,
            "Contrato": contrato_filtrado.to_dict(),
            "Cluster": {
                "Número": cluster,
                "Descripción": "El contrato pertenece al cluster número " + str(cluster)
            }
        }
        return response
    except KeyError:
        return {"error": "Índice de contrato no encontrado"}

@app.get("/cluster_info/{cluster}")
async def get_cluster_info(cluster: int):
    try:
        
        column_names = Base_Cluster_Est.drop(columns=['cluster']).columns.tolist()
        
        
        cluster_info = {
            "media_del_cluster": cluster_means.loc[cluster].to_dict(),
            "centroides": dict(zip(column_names, centroids[cluster].round(2).tolist())),
            }
        
        
        response = {
            "Cluster": {
                "Número": cluster,
                "Información del Cluster": cluster_info
            }
        }
        return response
    except KeyError:
        return {"error": "Cluster no encontrado"}

    
@app.get("/fila/{id_contrato}")
async def get_numero_fila(id_contrato: int):
    try:
        
        numero_fila = int(identificadores[identificadores['id_contrato'] == id_contrato]['numero_fila'])
        
        response = {
            "ID": id_contrato,
            "Número de fila": numero_fila,
            "Nota": f"Para encontrar el cluster correspondiente, escribe /indice/{numero_fila}"
        }
        return response
    except KeyError:
        return {"error": "ID de contrato no encontrado"}
    except ValueError:
        return {"error": "ID de contrato no encontrado"}

@app.get("/comparar_clusters")
async def compare_clusters():
    try:
        
        cluster_info = pd.DataFrame()

        
        for i in range(numero_clusters):
            cluster_info[f"Cluster {i}"] = cluster_means.loc[i]

        
        cluster_info = cluster_info.T

        
        cluster_info_dict = cluster_info.to_dict()

        
        response = {
            "Comparación de Clusters": cluster_info_dict
        }
        return response
    except KeyError:
        return {"error": "No se pudo realizar la comparación de clusters"}
    
if __name__ == "__main__":
    import uvicorn
    uvicorn.run( "APPI:app", host="127.0.0.1", port=8000) 

In [ ]:
#codigo bonito 
from fastapi import FastAPI, Form, Request
from fastapi.responses import HTMLResponse, JSONResponse
import pandas as pd
from sklearn.cluster import KMeans

app = FastAPI()

# datos 
Base_Cluster_Est = pd.read_csv("../../Datos/Raw/BaseDatos_Cluster.csv")
identificadores = pd.read_csv("../../Datos/Raw/identificadores.csv")
Base_filtrada = pd.read_csv("../../Datos/Raw/Base_filtrada.csv")


if 'Unnamed: 0' in Base_Cluster_Est.columns:
    Base_Cluster_Est.drop(columns=['Unnamed: 0'], inplace=True)

 
numero_clusters = 6
kmeans = KMeans(n_clusters=numero_clusters, random_state=42)
clusters = kmeans.fit_predict(Base_Cluster_Est)


Base_Cluster_Est['cluster'] = clusters

# Cálculo de las medias de cada cluster
cluster_means = Base_Cluster_Est.groupby('cluster').mean().round(2)


centroids = pd.DataFrame(kmeans.cluster_centers_).round(2)

# Combinación de los datos filtrados con los datos de los clusters
Base_Cluster_fusionada = Base_Cluster_Est.merge(Base_filtrada, left_index=True, right_index=True, how='right')

# HTML de la página principal
main_page_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>FastAPI App</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">¡Bienvenido!</h1>
        <form action="/execute_function" method="post">
            <div class="form-group">
                <label for="function">Selecciona la función:</label>
                <select class="form-control" id="function" name="function">
                    <option value="contract_info">Obtener Información de Contrato por ID</option>
                    <option value="cluster_info">Obtener Información de Cluster por Número de Cluster</option>
                    <option value="compare_clusters">Comparar las Características de los Clusters</option>
                </select>
            </div>
            <div class="form-group" id="contract_input" style="display: none;">
                <label for="id_contrato">Ingrese el ID del contrato:</label>
                <input type="text" class="form-control" id="id_contrato" name="id_contrato">
            </div>
            <div class="form-group" id="cluster_input" style="display: none;">
                <label for="cluster_number">Ingrese el número de cluster:</label>
                <input type="text" class="form-control" id="cluster_number" name="cluster_number">
            </div>
            <button type="submit" class="btn btn-primary">Seleccionar Función</button>
        </form>
    </div>

    <script>
        function showContractInput() {
            document.getElementById('contract_input').style.display = 'block';
            document.getElementById('cluster_input').style.display = 'none';
        }

        function showClusterInput() {
            document.getElementById('cluster_input').style.display = 'block';
            document.getElementById('contract_input').style.display = 'none';
        }

        document.getElementById('function').addEventListener('change', function() {
            var selectedFunction = this.value;
            if (selectedFunction === 'contract_info') {
                showContractInput();
            } else if (selectedFunction === 'cluster_info') {
                showClusterInput();
            } else {
                document.getElementById('contract_input').style.display = 'none';
                document.getElementById('cluster_input').style.display = 'none';
            }
        });
    </script>
</body>
</html>
"""

#contrato por su ID
def get_contract_info(id_contrato):
    try:
        id_contrato = int(id_contrato)
        contrato = Base_Cluster_fusionada[Base_Cluster_fusionada['id_contrato'] == id_contrato].iloc[0]
        cluster = int(contrato['cluster'])
        contrato_filtrado = contrato[Base_filtrada.columns]
        return f"""
        <h1>Información del Contrato</h1>
        <table class="table">
            <tbody>
                <tr><th>ID del Contrato</th><td>{id_contrato}</td></tr>
                <tr><th>Contrato</th><td>{contrato_filtrado.to_html()}</td></tr>
                <tr><th>Cluster Número</th><td>{cluster}</td></tr>
                <tr><th>Descripción del Cluster</th><td>El contrato pertenece al cluster número {cluster}</td></tr>
            </tbody>
        </table>
        """
    except (KeyError, ValueError, IndexError):
        return "<h1>Error</h1><p>No se encontró información para el ID de contrato proporcionado.</p>"

# cluster
def get_cluster_info(cluster_number):
    try:
        cluster_number = int(cluster_number)
        column_names = Base_Cluster_Est.drop(columns=['cluster']).columns.tolist()
        cluster_info = {
            "media_del_cluster": cluster_means.loc[cluster_number].to_dict(),
            "centroides": dict(zip(column_names, centroids.iloc[cluster_number].tolist())),
        }
        return f"""
        <h1>Información del Cluster</h1>
        <table class="table">
            <tbody>
                <tr><th>Cluster Número</th><td>{cluster_number}</td></tr>
                <tr><th>Información del Cluster</th><td>{pd.DataFrame(cluster_info).to_html()}</td></tr>
            </tbody>
        </table>
        """
    except KeyError:
        return "<h1>Error</h1><p>No se encontró el cluster especificado.</p>"

# comparar 
def compare_clusters():
    try:
        return f"""
        <h1>Comparación de Clusters</h1>
        {cluster_means.to_html()}
        """
    except KeyError:
        return "<h1>Error</h1><p>No se pudo realizar la comparación de clusters.</p>"


@app.get("/", response_class=HTMLResponse)
async def root():
    return main_page_html

# Rutas
@app.post("/execute_function", response_class=HTMLResponse)
async def execute_function(request: Request, function: str = Form(...), id_contrato: str = Form(None), cluster_number: str = Form(None)):
    if function == "contract_info" and id_contrato:
        return get_contract_info(id_contrato)
    elif function == "cluster_info" and cluster_number:
        return get_cluster_info(cluster_number)
    elif function == "compare_clusters":
        return compare_clusters()
    else:
        return main_page_html

#uvicon
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


In [3]:
from fastapi import FastAPI
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import logging

Base_Cluster

In [4]:
Base_Cluster_Est = pd.read_csv("../../Datos/Raw/BaseDatos_Cluster.csv")
indices = pd.read_csv("../../Datos/Raw/identificadores.csv")
Base_filtrada = pd.read_csv("../../Datos/Raw/Base_filtrada.csv")

In [11]:
Base_Cluster_fusionada = Base_Cluster_Est.merge(Base_filtrada, left_index=True, right_index=True)
Base_Cluster_fusionada.columns

Index(['año_firma', 'mes_firma', 'año_inicio', 'mes_inicio', 'año_fin',
       'mes_fin', 'estado_contrato_Activo', 'estado_contrato_Cerrado',
       'estado_contrato_En ejecución', 'estado_contrato_Modificado',
       ...
       'liquidaci_n', 'obligaci_n_ambiental', 'obligaciones_postconsumo',
       'reversion', 'origen_de_los_recursos', 'destino_gasto',
       'valor_del_contrato', 'mes_anio', 'duracion_del_contrato',
       'log_valor_contrato'],
      dtype='object', length=103)

In [12]:
numero_clusters = 6
kmeans = KMeans(n_clusters=numero_clusters, random_state=42)
clusters = kmeans.fit_predict(Base_Cluster_Est)

# Añadir una nueva columna al DataFrame con el número de cluster para cada fila
Base_Cluster_Est['cluster'] = clusters

In [15]:
Base_Cluster_Est

,año_firma,mes_firma,año_inicio,mes_inicio,año_fin,mes_fin,estado_contrato_Activo,estado_contrato_Cerrado,estado_contrato_En ejecución,estado_contrato_Modificado,...,obligaci_n_ambiental_No,obligaci_n_ambiental_Si,origen_de_los_recursos_No Definido,origen_de_los_recursos_Presupuesto de entidad nacional,origen_de_los_recursos_Regalías,destino_gasto_Funcionamiento,destino_gasto_Inversión,destino_gasto_No Definido,valor_del_contrato_estandarizado,cluster
id_contrato,,,,,,,,,,,,,,,,,,,,,
4696608,2023,2,2023,3,2024,7,False,False,True,False,...,True,False,True,False,False,True,False,False,-0.008631,4
5719533,2024,1,2024,1,2024,12,False,False,False,True,...,True,False,True,False,False,False,True,False,-0.012515,4
2647985,2021,7,2021,7,2021,11,False,False,True,False,...,True,False,True,False,False,False,True,False,-0.049259,1
3423229,2022,1,2022,1,2022,12,False,False,True,False,...,True,False,True,False,False,False,True,False,-0.037764,4
5849002,2024,2,2024,2,2024,12,False,False,True,False,...,True,False,True,False,False,False,True,False,-0.039567,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360959,2022,1,2022,1,2022,9,False,False,True,False,...,True,False,True,False,False,False,True,False,-0.053837,4
3203258,2022,1,2022,1,2022,12,False,False,False,True,...,True,False,True,False,False,False,True,False,-0.024247,4
5951068,2024,2,2024,2,2024,9,False,False,True,False,...,True,False,True,False,False,False,True,False,-0.035430,4
